In [1]:
import pandas as pd
import numpy as np 
import numpy.random as rd
from sklearn.model_selection import train_test_split
import random

pd.set_option('display.max_columns',None)

In [2]:
df_home=pd.read_csv('/Users/aurelientarroux/Desktop/Data_Challenge_QRT/Data/Train_Data/train_home_team_statistics_df.csv')
df_away=pd.read_csv('/Users/aurelientarroux/Desktop/Data_Challenge_QRT/Data/Train_Data/train_away_team_statistics_df.csv')

df_home.columns='home_'+df_home.columns
df_away.columns='away_'+df_away.columns

df_result=pd.read_csv('/Users/aurelientarroux/Desktop/Data_Challenge_QRT/Data/Y_train_1rknArQ.csv')

base_match=pd.merge(df_home,df_away,left_on="home_ID",right_on='away_ID',how='inner')
base=pd.merge(base_match,df_result,left_on="home_ID",right_on='ID',how='inner')
base=base.drop(['home_ID','home_LEAGUE','home_TEAM_NAME','away_ID','away_LEAGUE','away_TEAM_NAME'],axis=1)

def resultat(row):
    if row['HOME_WINS']==1:
        return 'HOME'
    elif row['AWAY_WINS']==1:
        return 'AWAY'
    else:
        return 'DRAW'

base['Resultat']=base.apply(resultat, axis=1)
base=base.drop(['HOME_WINS','AWAY_WINS','DRAW'],axis=1)

base.dropna(inplace=True)

X = base.drop(['Resultat','ID'],axis=1)
y = base['Resultat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train

,home_TEAM_SHOTS_TOTAL_season_sum,home_TEAM_SHOTS_INSIDEBOX_season_sum,home_TEAM_SHOTS_OFF_TARGET_season_sum,home_TEAM_SHOTS_ON_TARGET_season_sum,home_TEAM_SHOTS_OUTSIDEBOX_season_sum,home_TEAM_PASSES_season_sum,home_TEAM_SUCCESSFUL_PASSES_season_sum,home_TEAM_SAVES_season_sum,home_TEAM_CORNERS_season_sum,home_TEAM_FOULS_season_sum,home_TEAM_YELLOWCARDS_season_sum,home_TEAM_REDCARDS_season_sum,home_TEAM_OFFSIDES_season_sum,home_TEAM_ATTACKS_season_sum,home_TEAM_PENALTIES_season_sum,home_TEAM_SUBSTITUTIONS_season_sum,home_TEAM_BALL_SAFE_season_sum,home_TEAM_DANGEROUS_ATTACKS_season_sum,home_TEAM_INJURIES_season_sum,home_TEAM_GOALS_season_sum,home_TEAM_GAME_WON_season_sum,home_TEAM_GAME_DRAW_season_sum,home_TEAM_GAME_LOST_season_sum,home_TEAM_SHOTS_TOTAL_season_average,home_TEAM_SHOTS_INSIDEBOX_season_average,home_TEAM_SHOTS_OFF_TARGET_season_average,home_TEAM_SHOTS_ON_TARGET_season_average,home_TEAM_SHOTS_OUTSIDEBOX_season_average,home_TEAM_PASSES_season_average,home_TEAM_SUCCESSFUL_PASSES_season_average,home_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average,home_TEAM_SAVES_season_average,home_TEAM_CORNERS_season_average,home_TEAM_BALL_POSSESSION_season_average,home_TEAM_FOULS_season_average,home_TEAM_YELLOWCARDS_season_average,home_TEAM_REDCARDS_season_average,home_TEAM_OFFSIDES_season_average,home_TEAM_ATTACKS_season_average,home_TEAM_PENALTIES_season_average,home_TEAM_SUBSTITUTIONS_season_average,home_TEAM_BALL_SAFE_season_average,home_TEAM_DANGEROUS_ATTACKS_season_average,home_TEAM_INJURIES_season_average,home_TEAM_GOALS_season_average,home_TEAM_GAME_WON_season_average,home_TEAM_GAME_DRAW_season_average,home_TEAM_GAME_LOST_season_average,home_TEAM_SHOTS_TOTAL_season_std,home_TEAM_SHOTS_INSIDEBOX_season_std,home_TEAM_SHOTS_OFF_TARGET_season_std,home_TEAM_SHOTS_ON_TARGET_season_std,home_TEAM_SHOTS_OUTSIDEBOX_season_std,home_TEAM_PASSES_season_std,home_TEAM_SUCCESSFUL_PASSES_season_std,home_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std,home_TEAM_SAVES_season_std,home_TEAM_CORNERS_season_std,home_TEAM_BALL_POSSESSION_season_std,home_TEAM_FOULS_season_std,home_TEAM_YELLOWCARDS_season_std,home_TEAM_REDCARDS_season_std,home_TEAM_OFFSIDES_season_std,home_TEAM_ATTACKS_season_std,home_TEAM_PENALTIES_season_std,home_TEAM_SUBSTITUTIONS_season_std,home_TEAM_BALL_SAFE_season_std,home_TEAM_DANGEROUS_ATTACKS_season_std,home_TEAM_INJURIES_season_std,home_TEAM_GOALS_season_std,home_TEAM_SHOTS_TOTAL_5_last_match_sum,home_TEAM_SHOTS_INSIDEBOX_5_last_match_sum,home_TEAM_SHOTS_OFF_TARGET_5_last_match_sum,home_TEAM_SHOTS_ON_TARGET_5_last_match_sum,home_TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum,home_TEAM_PASSES_5_last_match_sum,home_TEAM_SUCCESSFUL_PASSES_5_last_match_sum,home_TEAM_SAVES_5_last_match_sum,home_TEAM_CORNERS_5_last_match_sum,home_TEAM_FOULS_5_last_match_sum,home_TEAM_YELLOWCARDS_5_last_match_sum,home_TEAM_REDCARDS_5_last_match_sum,home_TEAM_OFFSIDES_5_last_match_sum,home_TEAM_ATTACKS_5_last_match_sum,home_TEAM_PENALTIES_5_last_match_sum,home_TEAM_SUBSTITUTIONS_5_last_match_sum,home_TEAM_BALL_SAFE_5_last_match_sum,home_TEAM_DANGEROUS_ATTACKS_5_last_match_sum,home_TEAM_INJURIES_5_last_match_sum,home_TEAM_GOALS_5_last_match_sum,home_TEAM_GAME_WON_5_last_match_sum,home_TEAM_GAME_DRAW_5_last_match_sum,home_TEAM_GAME_LOST_5_last_match_sum,home_TEAM_SHOTS_TOTAL_5_last_match_average,home_TEAM_SHOTS_INSIDEBOX_5_last_match_average,home_TEAM_SHOTS_OFF_TARGET_5_last_match_average,home_TEAM_SHOTS_ON_TARGET_5_last_match_average,home_TEAM_SHOTS_OUTSIDEBOX_5_last_match_average,home_TEAM_PASSES_5_last_match_average,home_TEAM_SUCCESSFUL_PASSES_5_last_match_average,home_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average,home_TEAM_SAVES_5_last_match_average,home_TEAM_CORNERS_5_last_match_average,home_TEAM_BALL_POSSESSION_5_last_match_average,home_TEAM_FOULS_5_last_match_average,home_TEAM_YELLOWCARDS_5_last_match_average,home_TEAM_REDCARDS_5_last_match_average,home_TEAM_OFFSIDES_5_last_match_average,home_TEAM_ATTACKS_5_last_match_average,home_TEAM_PENALTI

In [3]:
n_arbre=100
n_sample=len(X)
n_carac=int(np.sqrt(len(X.columns)))
colonne=list(X.columns)

In [4]:
def calcule_gini(carac,individus):
    liste_Gini=[]
    nb_individus=len(individus)
    for k in list(carac): #je traite le calcule de gini pour chaque caractéristiques 
    
        carac_traite=k
        df_interess=pd.DataFrame(X_train.loc[list(individus),carac])

        #je détermine les différents seuils possible
        a=list(df_interess.loc[:,carac_traite])
        liste_distincte = list(set(a))
        liste_distincte.sort()
        liste_moy=[]
        for j in range(len(liste_distincte)-1):
            liste_moy+= [(liste_distincte[j+1]+liste_distincte[j])/2]

        #je calcule l'indice de Gini de la caractéritsique pour chaque nombre de liste_moy et faut que m'assure que l'index soit l'id match
        for i in liste_moy:

            index_inf=df_interess[df_interess[carac_traite]<i].index
            index_sup=df_interess[df_interess[carac_traite]>i].index

            
            df_inf=y_train.loc[list(index_inf)]
            df_sup=y_train.loc[list(index_sup)]

            nb_inf=(df_inf.value_counts()/len(df_inf))**2
            nb_sup=(df_sup.value_counts()/len(df_sup))**2

            Gini_1=0
            Gini_2=0
            for k in nb_inf:
                Gini_1+=k
            for k in nb_sup:
                Gini_2+=k
            Gini_1=1-Gini_1
            Gini_2=1-Gini_2

            Gini_split= ((len(index_inf)/nb_individus)*Gini_1) + ((len(index_sup)/nb_individus)*Gini_2)
            
            liste_Gini+=[[Gini_split,i,carac_traite]]

    liste_Gini.sort(key=lambda x: x[0])  

    carac_chosie=liste_Gini[0][2]
    seuil=liste_Gini[0][1]

    return  [carac_chosie, seuil]



In [6]:
colonne=list(X_train.columns)
individus_tot=X_train.index
print(individus_tot)
nombre_arbre=100
foret=[]


for k in range(nombre_arbre):
    
    arbre=[]
    liste_liste_individus=[list(individus_tot)]
    n = len(liste_liste_individus)
    liste_individus = random.choices(liste_liste_individus, k=n)
    
    liste_carac=[[0]]
    while len(liste_individus)>0:
        individus_inter=liste_individus[0]
        
        if len(individus_inter)>10:
            
            carac=rd.choice(colonne, size=n_carac, replace=True)
            carac_choisie , seuil =calcule_gini(carac,individus_inter)
            df_inter=X_train.loc[list(individus_inter),:]
            individus_gauche=df_inter[df_inter[carac_choisie]<seuil].index
            individus_droite=df_inter[df_inter[carac_choisie]>seuil].index
            
            liste_individus+=[list(individus_gauche),list(individus_droite)]
            
            nouvelle_carac_gauche = liste_carac[0] + [[carac_choisie, seuil, '<']]
            nouvelle_carac_droite = liste_carac[0] + [[carac_choisie, seuil, '>']]
            liste_carac += [nouvelle_carac_gauche, nouvelle_carac_droite]
            
        else:
            top_resultat=((y_train.loc[individus_inter].value_counts()).sort_values(ascending=False).index)[0]
            arbre+=[[top_resultat,liste_carac[0]]]

        del liste_individus[0]
        del liste_carac[0]
    
    foret+=[[arbre]]
    print(k)


Index([ 6596,  9983, 10028,  5045,  5116,  5000,  2585,  7328, 10607,  7386,
       ...
        2510,  8308,  6616,   718,  9329,  8539,  7740,  8030,  1283, 10793],
      dtype='int64', length=6610)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [8]:
colonne=list(X_train.columns)
individus_tot=X_test.index
compte_ratio_final=0

for k in range(len(individus_tot)):
    liste_resultat_final=[]
    individus_test=X_test.iloc[k,:]
    for arbre in foret:
        for branche in arbre[0]:
            resultat=branche[0]
            compte_bon=0
            compte_neoud=0
            
            for neoud in branche[1]:
                if neoud!=0:
                    compte_neoud+=1
                    
                    carac=neoud[0]
                    seuil=neoud[1]
                    signe=neoud[2]
                    
                    carac_individus=individus_test[carac]
                    
                    if signe=='<':
                        if carac_individus<seuil:
                            compte_bon+=1
                    
                    elif signe=='>':
                        if carac_individus>seuil:
                            compte_bon+=1
                    else:
                        print('merde')
            
            if compte_bon==compte_neoud:
                liste_resultat_final+=[resultat]
                break

    if max(liste_resultat_final, key=liste_resultat_final.count)==y_test.iloc[k]:
        compte_ratio_final+=1
    print(compte_ratio_final/(k+1))


0.0
0.0
0.0
0.25
0.2
0.3333333333333333
0.42857142857142855
0.5
0.5555555555555556
0.6
0.5454545454545454
0.5
0.5384615384615384
0.5
0.5333333333333333
0.5
0.47058823529411764
0.4444444444444444
0.42105263157894735
0.4
0.42857142857142855
0.45454545454545453
0.43478260869565216
0.4166666666666667
0.4
0.4230769230769231
0.4074074074074074
0.42857142857142855
0.4482758620689655
0.4666666666666667
0.4838709677419355
0.5
0.5151515151515151
0.5
0.5142857142857142
0.5
0.5135135135135135
0.5263157894736842
0.5128205128205128
0.5
0.5121951219512195
0.5238095238095238
0.5116279069767442
0.5
0.4888888888888889
0.5
0.48936170212765956
0.4791666666666667
0.4897959183673469
0.48
0.47058823529411764
0.46153846153846156
0.4528301886792453
0.4444444444444444
0.43636363636363634
0.42857142857142855
0.43859649122807015
0.43103448275862066
0.4406779661016949
0.43333333333333335
0.4262295081967213
0.43548387096774194
0.4444444444444444
0.4375
0.4307692307692308
0.4393939393939394
0.43283582089552236
0.441

In [1]:
print(compte_ratio_final)

NameError: name 'compte_ratio_final' is not defined